In [1]:
import os, sys
from ipyparallel import Client
import matplotlib.pyplot as plt
%matplotlib inline
from subprocess import Popen, PIPE
from Bio import SeqIO
import pandas as pd

In [2]:
#get fastq files
qdir = '/home/lindb/wbp/concatenated/'
files = os.listdir(qdir)
files = [os.path.join(qdir,f) for f in files if f.endswith('d.fastq')]
len(files)

244

In [1035]:
rc = Client(profile="xmn")
dview = rc[:]
lview = rc.load_balanced_view()
print(len(rc))

10


In [594]:
assembly = '/home/lindb/wbp/concatenated/velvet/auto_data_45/contigs.fa'

In [5]:
cd ~/wbp/concatenated/raw_mapping/

/gpfs_fs/home/lindb/wbp/concatenated/raw_mapping


In [6]:
!/home/lindb/g/src/bowtie2-2.2.6/bowtie2-build -f $assembly $assembly

Settings:
  Output files: "/home/lindb/wbp/concatenated/velvet/auto_data_45/contigs.fa.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /home/lindb/wbp/concatenated/velvet/auto_data_45/contigs.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 12226508
Using parameters --bmax 9169881 --dcv 1024
  Doing ahead-of-time memory usage test
  Pa

In [105]:
# --very-fast-local
# Same as: -D 5 -R 1 -N 0 -L 25 -i S,1,2.00

# --fast-local
# Same as: -D 10 -R 2 -N 0 -L 22 -i S,1,1.75

# --sensitive-local
# Same as: -D 15 -R 2 -N 0 -L 20 -i S,1,0.75 (default in --local mode)

# --very-sensitive-local
# Same as: -D 20 -R 3 -N 0 -L 20 -i S,1,0.50

@lview.remote()
def run_bowtie2(args):
    import os, stopwatch, multiprocessing
    timer = stopwatch.Timer()
    cpus = multiprocessing.cpu_count()
    assembly, reads = args
    parent = os.path.dirname(reads)
    outdir = os.path.join(parent, "raw_mapping_samfiles")
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    sam = os.path.join(outdir, "%s.sam" % (os.path.basename(reads)))
    cmd = "/home/lindb/g/src/bowtie2-2.2.6/bowtie2 --local -D 20 -R 3 -N 1 -L 20 -i S,1,0.50 -p %d -x %s -U %s -S %s" % (cpus,
                                                               assembly,
                                                               reads,
                                                               sam)
    res = None
    res = !$cmd
#     if not os.path.exists(sam):
#         res = !$cmd
    timer.stop()
    return assembly, sam, cmd, timer.elapsed, res

In [107]:
#make sure they're the right files
for f in files:
    print f
    break

/home/lindb/wbp/concatenated/055compiled.fastq


In [106]:
len(files)

244

In [108]:
#map with bowtie2
bowtie_jobs = []
for f in files:
    bowtie_jobs.append(run_bowtie2((assembly, f)))

In [248]:
#check on progress
count = 0
for j in bowtie_jobs:
    if j.ready():
        count += 1
print count,'/',len(bowtie_jobs)

244 / 244


In [234]:
bowtie_jobs

[<AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: run_bowtie2>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: run_bowtie2>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: run_bowtie2>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <AsyncResult: finished>,
 <A

In [192]:
from io import StringIO

In [249]:
#make sure there are no errors!
count = 0
ecount = 0
for j in bowtie_jobs:
    if j.ready():
        try:
            res = [x.strip() for x in j.r[-1]]
        except Exception as E:
            print "%s engine died" % os.path.basename(files[count]).split(".")[0]
            ecount += 1
            #print E,'\n'
        #print files[count].split("/")[-1]
        #print res
        res = ""
    count += 1
ecount

119compiled engine died
117compiled engine died
039compiled engine died
030compiled engine died
256compiled engine died
250compiled engine died
163compiled engine died
091compiled engine died
015compiled engine died
066compiled engine died
153compiled engine died


11

In [267]:
#look at overall mapping percentages across samples
overall = []
for j in bowtie_jobs:
    if j.ready():
        try:
            path = j.r[1]
            res_file = "%s.stats" % path
            res = j.r[-1]
            res = [x.strip() for x in res]
            percent = res[-1].split()[0][:-1]
            overall.append(float(percent))
        except Exception as E:
            pass
pd.Series(overall).describe()

count    233.000000
mean      84.347983
std       10.083955
min       38.010000
25%       78.330000
50%       84.710000
75%       92.510000
max       99.410000
dtype: float64

In [290]:
#add in 11 that failed above because engine died; I had to run in another window
sec_overall = [85.83, 92.78, 80.02, 76.73, 94.65, 91.83, 94.56, 78.43, 88.95, 99.24, 93.76]
len(sec_overall)

11

In [283]:
test = sec_overall + overall
len(test)

244

In [285]:
len(overall)

233

In [286]:
overall

[76.62,
 84.98,
 77.61,
 77.07,
 86.26,
 80.12,
 81.82,
 93.84,
 94.26,
 82.89,
 50.61,
 93.6,
 91.51,
 90.77,
 93.08,
 76.35,
 93.2,
 46.83,
 94.55,
 91.17,
 95.73,
 77.14,
 93.33,
 76.49,
 76.86,
 96.43,
 95.66,
 93.52,
 77.6,
 79.19,
 81.85,
 93.63,
 93.52,
 96.42,
 92.2,
 93.23,
 55.28,
 92.48,
 87.34,
 83.3,
 77.18,
 80.7,
 77.99,
 91.59,
 82.69,
 78.33,
 40.24,
 90.68,
 79.17,
 50.83,
 81.04,
 97.13,
 81.74,
 70.2,
 93.76,
 95.17,
 92.94,
 91.21,
 96.13,
 91.58,
 76.61,
 78.97,
 90.39,
 79.86,
 83.53,
 88.18,
 77.33,
 79.23,
 76.58,
 93.08,
 47.25,
 93.56,
 77.08,
 94.15,
 84.77,
 75.41,
 70.71,
 93.17,
 93.28,
 76.48,
 84.71,
 95.13,
 96.86,
 78.15,
 72.2,
 86.15,
 86.73,
 82.01,
 85.83,
 77.63,
 83.11,
 82.37,
 76.73,
 96.62,
 92.06,
 78.73,
 94.08,
 78.96,
 92.39,
 85.28,
 95.91,
 84.64,
 79.02,
 76.24,
 38.01,
 75.7,
 96.2,
 83.03,
 90.46,
 99.24,
 76.61,
 76.13,
 92.38,
 99.02,
 92.09,
 92.73,
 77.37,
 85.03,
 75.13,
 92.57,
 91.84,
 93.39,
 95.56,
 96.08,
 92.41,
 81.7,
 93

In [289]:
for x in overall:
    if x not in test:
        print "crap"
for y in sec_overall:
    if y not in test:
        print 'crap'

In [291]:
pd.Series(test).describe()

count    244.000000
mean      84.548607
std       10.013239
min       38.010000
25%       78.427500
50%       84.770000
75%       92.625000
max       99.410000
dtype: float64

In [ ]:
#output from pd.Series(test).describe()
#count    244.000000
#mean      84.548607
#std       10.013239
#min       38.010000
#25%       78.427500
#50%       84.770000
#75%       92.625000
#max       99.410000
#dtype: float64

# convert sam to bam

In [46]:
#get all of the *.sam files
DIR = '/home/lindb/wbp/concatenated/raw_mapping_samfiles/'
sam_files = os.listdir(DIR)
sam_files = [os.path.join(DIR,f) for f in sam_files if f.endswith('sam')]
len(sam_files)

244

In [70]:
#example
sam_files[0]

'/home/lindb/wbp/concatenated/raw_mapping_samfiles/218compiled.fastq.sam'

In [75]:
sorted(sam_files)

['/home/lindb/wbp/concatenated/raw_mapping_samfiles/009compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/010compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/011compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/012compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/013compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/014compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/015compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/016compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/017compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/018compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/019compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/020compiled.fastq.sam',
 '/home/lindb/wbp/concatenated/raw_mapping_samfiles/021compiled.fastq.sam',
 '/home/lind

In [76]:
#to make sure function will name like I want it to
sam = sam_files[0]
bam = sam.replace(".sam", ".bam")
bam_sorted = "%s_sorted.bam" % bam.replace(".bam", "")
bam_index = bam_sorted.replace(".bam", ".bai")
print sam
print bam
print bam_sorted
print bam_index
del sam,bam,bam_sorted,bam_index

/home/lindb/wbp/concatenated/raw_mapping_samfiles/218compiled.fastq.sam
/home/lindb/wbp/concatenated/raw_mapping_samfiles/218compiled.fastq.bam
/home/lindb/wbp/concatenated/raw_mapping_samfiles/218compiled.fastq_sorted.bam
/home/lindb/wbp/concatenated/raw_mapping_samfiles/218compiled.fastq_sorted.bai


In [50]:
@lview.remote()
def convert_sam_to_bam(sam):
    import stopwatch, multiprocessing, os
    timer = stopwatch.Timer()
    cpus = multiprocessing.cpu_count()
    bam = sam.replace(".sam", ".bam")
    bam_sorted = "%s_sorted.bam" % bam.replace(".bam", "")
    bam_index = bam_sorted.replace(".bam", ".bai")
    if not os.path.exists(bam):
        !samtools view -bS $sam > $bam
#    if os.path.exists(bam):
#        if not os.path.exists(bam_sorted):
#            !samtools sort -@ $cpus $bam -o $bam_sorted
#    if os.path.exists(bam_sorted):
#        if not os.path.exists(bam_index):
#            !samtools index $bam_sorted $bam_index
    timer.stop()
    return bam, bam_sorted, bam_index, timer.elapsed

In [51]:
#convert *.sam to *.bam
sam_bam_jobs = []
for f in sam_files:
    sam_bam_jobs.append(convert_sam_to_bam(os.path.abspath(f)))

In [201]:
count = 0
for j in sam_bam_jobs:
    if j.ready():
        count += 1
count,len(sam_bam_jobs)        

(244, 244)

In [202]:
#make sure there are no errors!
count = 0
ecount = 0
rcount = 0
for j in sam_bam_jobs:
    if j.ready():
        rcount += 1
        try:
            hey = j.r
            #print j.r
        except Exception as E:
            print "%s engine died" % os.path.basename(sam_files[count]).split(".")[0]
            print E, '\n'
            ecount += 1
            #print E,'\n'
        #print files[count].split("/")[-1]
        #print res
        res = ""
    count += 1
ecount, rcount

(0, 244)

In [229]:
#get bam files
bam_files = os.listdir(DIR)
bam_files = [os.path.join(DIR,f) for f in bam_files if f.endswith('q.bam')]
len(bam_files)

244

In [252]:
count = 0
for bam in bam_files:
    bam_sorted = "%s_sorted.bam" % bam.replace(".bam", "")
    if not os.path.exists(bam_sorted):        
        num = os.path.basename(bam)[:3]
        text = '''#!/bin/bash
#$ -N run%s
#$ -V
#$ -j y
#$ -cwd

cd /home/lindb/wbp/concatenated/raw_mapping_samfiles/
samtools sort -@ 24 %s -o %s
''' % (num,bam,bam_sorted)
        f = "sort%s.sh" % str(num)
        filE = '/home/lindb/wbp/concatenated/raw_mapping_samfiles/sortrunfiles/%s' % f
        with open(filE,'w') as o:
            o.write("%s" % text)    

In [294]:
@lview.remote()
def index_sorted_bams(bam_sorted):
    import stopwatch, multiprocessing, os
    timer = stopwatch.Timer()
    cpus = multiprocessing.cpu_count() 
    bam_index = bam_sorted.replace(".bam", ".bai")
    if not os.path.exists(bam_index):
        !samtools index $bam_sorted $bam_index    
    timer.stop()
    return bam_sorted, bam_index, timer.elapsed            

In [302]:
sorted_bams = os.listdir(DIR)
sorted_bams = [os.path.join(DIR,f) for f in sorted_bams if 'sorted.bam' in f]
len(sorted_bams)

244

In [304]:
for bam_sorted in sorted_bams:
    print bam_sorted
    break

/home/lindb/wbp/concatenated/raw_mapping_samfiles/203compiled.fastq_sorted.bam


In [306]:
count = 0
for bam_sorted in sorted_bams:
    bam_index = bam_sorted.replace(".bam", ".bai")
    if not os.path.exists(bam_index):        
        num = os.path.basename(bam_sorted)[:3]
        text = '''#!/bin/bash
#$ -N run%s
#$ -V
#$ -j y
#$ -cwd

cd /home/lindb/wbp/concatenated/raw_mapping_samfiles/
samtools index %s %s
''' % (num,bam_sorted,bam_index)
        f = "index%s.sh" % str(num)
        filE = '/home/lindb/wbp/concatenated/raw_mapping_samfiles/sortrunfiles/%s' % f
        with open(filE,'w') as o:
            o.write("%s" % text)    

# add rg info

In [307]:
from collections import OrderedDict

In [308]:
#get files containing library and barcode IDs for each sample
bdir = '/home/lindb/wbp/raw_reads/barcodefiles/' 
bfiles = os.listdir(bdir)
bfiles = [os.path.join(bdir,f) for f in bfiles]
barfiles = []
libs = []
for f in bfiles:
    lib = os.path.basename(f).split("_")[0]
    if lib not in libs:
        libs.append(lib)
        barfiles.append(f)
barfiles

['/home/lindb/wbp/raw_reads/barcodefiles/1_1_150bp_barcodes.txt',
 '/home/lindb/wbp/raw_reads/barcodefiles/2_1_150bp_barcodes.txt',
 '/home/lindb/wbp/raw_reads/barcodefiles/3_1_150bp_barcodes.txt']

In [309]:
#grab libID and barcodeID for each sample from the files
badids = ['NA','nan','5_b11','8_a3','8_a4','8_a6','8_a7','2_a7','2_a9','5_b11','5_b9']
rgDict = OrderedDict()
for f in barfiles:
    df = pd.read_csv(f,header=0,sep="\t")
    lib = "lib%s" % str(os.path.basename(f).split("_")[0])
    for row in df.index:
        samp = str(df.loc[row,'sample.ID'])
        while len(samp) < 3:
            samp = '0%s' % samp
        if samp not in badids:
            rgDict[samp] = OrderedDict()
            bc = df.loc[row,'barcode2'].replace("CTCTTTCCCTACACGACGCTCTTCCGATCT", "")[:-1]
            #print samp,bc
            rgDict[samp]['lib'] = lib
            rgDict[samp]['bc'] = bc
len(rgDict.keys())

244

In [310]:
bcs = []
for samp in rgDict.keys():
    if rgDict[samp]['bc'] not in bcs:
        bcs.append(rgDict[samp]['bc'])
len(bcs)    #should be 96!!!!    

96

In [311]:
#take a look to see how function will work
bam = '/home/lindb/wbp/concatenated/raw_mapping_samfiles/218compiled.fastq_sorted.bam'
base = os.path.basename(bam).split(".")
samp = base[0][:3]
bam_rg = bam.replace(".bam", "_rg.bam")
rglb = rgDict[samp]['lib'] #lib ID
rgpu = rgDict[samp]['bc']  #barcode ID
rgsm = base[0]
rgid = "_".join([rglb,rgsm,rgpu])
print "base=",base
print "bam_rg=",bam_rg
print "rglb=",rglb #population ID
print 'rgpu=',rgpu
print 'rgsm =',rgsm
print 'rgid=',rgid
del bam, base,samp,bam_rg,rglb,rgpu,rgsm,rgid

base= ['218compiled', 'fastq_sorted', 'bam']
bam_rg= /home/lindb/wbp/concatenated/raw_mapping_samfiles/218compiled.fastq_sorted_rg.bam
rglb= lib1
rgpu= ggtaggaa
rgsm = 218compiled
rgid= lib1_218compiled_ggtaggaa


In [958]:
@lview.remote()
def add_rg_info_to_bam(args):
    rgDict , bam = args
    import os
    cmd = "java -jar /home/lindb/g/src/picard-tools-1.130/picard.jar AddOrReplaceReadGroups"
    base = os.path.basename(bam).split(".")
    samp = base[0][:3]
    bam_rg = bam.replace(".bam", "_rg.bam")
    rglb = rgDict[samp]['lib']    #libID
    rgpu = rgDict[samp]['bc']     #barcodeID
    rgsm = base[0]                #sampID
    rgid = "_".join([rglb,rgsm,rgpu])  #sampID_barcodeID
    rg_string = "RGID=%s RGLB=%s RGPL=illumina RGPU=%s RGSM=%s" % (rgid,
                                                                   rglb,
                                                                   rgpu,
                                                                   rgsm)
    cmd = "%s INPUT=%s OUTPUT=%s %s CREATE_INDEX=true" % (cmd,bam,bam_rg,rg_string)
    print cmd
    if not os.path.exists(bam_rg):
        !$cmd
    return bam_rg, rg_string, cmd

In [360]:
#get sorted.bam files
DIR = '/home/lindb/wbp/concatenated/raw_mapping_samfiles/'
sorted_bams = os.listdir(DIR)
sorted_bams = [os.path.join(DIR,f) for f in sorted_bams if 'sorted.bam' in f]
len(sorted_bams)

244

In [960]:
rg_lst = []
for f in sorted_bams:
    rg_lst.append(add_rg_info_to_bam([rgDict,f]))

In [1072]:
rcount = 0
for j in rg_lst:
    if j.ready():
        rcount += 1
rcount,len(rg_lst)

(244, 244)


In [1071]:
for f in sorted_bams:
    rg_bai = f.replace(".bam","_rg.bai")
    rg_bam = rg_bai.replace(".bai",".bam")
    if not os.path.exists(rg_bai):
        print 'bai',f
    if not os.path.exists(rg_bam):
        print 'bam',f